In [ ]:
# 聚类分析笔记本

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
import sys
sys.path.append('../')
from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor
from src.models.cluster_analyzer import ClusterAnalyzer
from src.models.pca_analyzer import PCAAnalyzer
from src.visualization.plot_utils import VisualizationUtils

# 1. 数据准备
print("=== 聚类分析 ===")

# 加载数据
loader = DataLoader()
panel_data = loader.load_panel_data()

# 使用2023年数据
data_2023 = panel_data[panel_data['年份'] == 2023].copy()

# 选择聚类变量
cluster_vars = [
    '跨境数据传输总量_TB',
    '数据中心数量',
    '互联网国际出口带宽_Gbps',
    'GDP_亿元',
    '数字经济核心产业增加值_亿元',
    '研发经费投入_亿元',
    '5G基站数量',
    '算力规模_PFLOPS'
]

# 检查变量是否存在
existing_vars = [var for var in cluster_vars if var in data_2023.columns]
print(f"使用的聚类变量: {len(existing_vars)}个")

# 提取数据
X_raw = data_2023[existing_vars].values
city_labels = data_2023['城市'].values
city_codes = data_2023['city_code'].values

print(f"数据形状: {X_raw.shape}")
print(f"城市数量: {len(city_labels)}")

# 2. 数据预处理
print("\n=== 数据预处理 ===")
preprocessor = DataPreprocessor(method='median', scale_method='standard')

# 处理缺失值
X_clean = preprocessor.handle_missing_values(pd.DataFrame(X_raw, columns=existing_vars))

# 处理异常值
X_no_outliers = preprocessor.handle_outliers(X_clean, method='cap')

# 标准化
X_scaled = preprocessor.scale_data(X_no_outliers)

print("预处理完成!")

# 3. 聚类分析
print("\n=== 聚类分析 ===")
cluster_analyzer = ClusterAnalyzer(random_state=42)

# 3.1 确定最优聚类数量
print("\n1. 确定最优聚类数量...")

# 使用K-means方法确定最优聚类数
kmeans_scores = cluster_analyzer.determine_optimal_clusters(
    X_scaled.values, 
    method='kmeans',
    max_clusters=min(8, len(city_labels))
)

print("K-means聚类评估:")
for i in range(len(kmeans_scores['n_clusters'])):
    print(f"  聚类数={kmeans_scores['n_clusters'][i]:2d}: "
          f"轮廓系数={kmeans_scores['silhouette'][i]:.3f}, "
          f"CH指数={kmeans_scores['calinski_harabasz'][i]:.1f}")

# 选择轮廓系数最大的聚类数
best_k_index = np.argmax(kmeans_scores['silhouette'])
optimal_n_clusters = kmeans_scores['n_clusters'][best_k_index]
print(f"\n最优聚类数量: {optimal_n_clusters}")

# 3.2 可视化聚类评估
print("\n2. 可视化聚类评估...")
cluster_analyzer.plot_cluster_evaluation(
    kmeans_scores, 
    method='kmeans',
    save_path='../outputs/figures/cluster_evaluation.png'
)

# 4. 应用不同聚类算法
print("\n=== 应用不同聚类算法 ===")

clustering_results = {}
methods = ['kmeans', 'hierarchical', 'gmm']

for method in methods:
    print(f"\n应用{method}聚类...")
    
    if method == 'kmeans':
        labels = cluster_analyzer.fit_clustering(
            X_scaled.values,
            method='kmeans',
            n_clusters=optimal_n_clusters
        )
    elif method == 'hierarchical':
        labels = cluster_analyzer.fit_clustering(
            X_scaled.values,
            method='hierarchical',
            n_clusters=optimal_n_clusters
        )
    elif method == 'gmm':
        labels = cluster_analyzer.fit_clustering(
            X_scaled.values,
            method='gmm',
            n_clusters=optimal_n_clusters
        )
    
    clustering_results[method] = labels
    
    # 计算聚类质量
    if len(np.unique(labels)) > 1:
        silhouette = cluster_analyzer.scores.get('silhouette', 0)
        print(f"  轮廓系数: {silhouette:.3f}")
        
        # 显示每个聚类的城市
        print(f"  聚类分布:")
        unique_labels = np.unique(labels)
        for label in unique_labels:
            if label != -1:  # 跳过噪声点
                cluster_cities = city_labels[labels == label]
                print(f"    聚类{label}: {len(cluster_cities)}个城市 - {', '.join(cluster_cities)}")

# 5. 选择最佳聚类结果
print("\n=== 选择最佳聚类结果 ===")
best_method = None
best_score = -1

for method, labels in clustering_results.items():
    if len(np.unique(labels)) > 1:
        score = silhouette_score(X_scaled.values, labels)
        if score > best_score:
            best_score = score
            best_method = method

print(f"最佳聚类方法: {best_method} (轮廓系数: {best_score:.3f})")
best_labels = clustering_results[best_method]

# 6. 聚类结果可视化
print("\n=== 聚类结果可视化 ===")

# 6.1 使用PCA降维可视化
print("1. PCA降维可视化...")
pca_visualizer = PCAAnalyzer(n_components=2)
pca_visualizer.fit(X_scaled.values, feature_names=existing_vars)
X_pca = pca_visualizer.transform(X_scaled.values)

# 绘制聚类散点图
fig, ax = plt.subplots(figsize=(12, 10))

unique_labels = np.unique(best_labels)
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))

for label, color in zip(unique_labels, colors):
    if label == -1:  # 噪声点
        color = 'gray'
        label_name = '噪声'
    else:
        label_name = f'聚类{label}'
    
    mask = best_labels == label
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
              c=[color], label=label_name,
              alpha=0.8, s=150, edgecolors='k')
    
    # 添加城市标签
    for i, city in enumerate(city_labels[mask]):
        ax.annotate(city, (X_pca[mask, 0][i], X_pca[mask, 1][i]),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=9, fontweight='bold')

ax.set_xlabel('PC1', fontsize=12)
ax.set_ylabel('PC2', fontsize=12)
ax.set_title(f'{best_method}聚类结果可视化 (PCA降维)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/figures/cluster_pca_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

# 6.2 绘制层次聚类树状图
print("2. 层次聚类树状图...")
if 'hierarchical' in clustering_results:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # 计算链接矩阵
    linkage_matrix = linkage(X_scaled.values, method='ward')
    
    # 绘制树状图
    dendrogram(linkage_matrix, labels=city_labels, ax=ax,
              leaf_rotation=90, leaf_font_size=10)
    
    ax.set_xlabel('城市', fontsize=12)
    ax.set_ylabel('距离', fontsize=12)
    ax.set_title('层次聚类树状图', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../outputs/figures/hierarchical_dendrogram.png', dpi=300, bbox_inches='tight')
    plt.show()

# 6.3 聚类特征分析
print("3. 聚类特征分析...")
cluster_stats = cluster_analyzer.analyze_cluster_characteristics(
    pd.DataFrame(X_scaled, columns=existing_vars),
    best_labels
)

print("\n聚类特征统计:")
print(cluster_stats.to_string(index=False))

# 7. 聚类结果综合展示
print("\n=== 聚类结果综合展示 ===")

# 创建聚类结果DataFrame
cluster_results_df = pd.DataFrame({
    '城市': city_labels,
    '城市代码': city_codes,
    '聚类标签': best_labels
})

# 添加原始特征值
for i, var in enumerate(existing_vars):
    cluster_results_df[var] = X_raw[:, i]

print("\n聚类结果:")
print(cluster_results_df[['城市', '聚类标签'] + existing_vars[:3]].to_string(index=False))

# 8. 聚类特征雷达图
print("\n=== 聚类特征雷达图 ===")

if cluster_analyzer.cluster_centers is not None:
    n_clusters = len(np.unique(best_labels[best_labels != -1]))
    n_features = min(6, len(existing_vars))
    
    # 选择关键特征
    selected_features = existing_vars[:n_features]
    
    # 计算每个聚类的特征均值（标准化前）
    cluster_means_original = []
    for label in np.unique(best_labels):
        if label != -1:  # 跳过噪声点
            mask = best_labels == label
            cluster_means = X_raw[mask][:, :n_features].mean(axis=0)
            cluster_means_original.append(cluster_means)
    
    if cluster_means_original:
        cluster_means_original = np.array(cluster_means_original)
        
        # 创建雷达图
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(projection='polar'))
        
        angles = np.linspace(0, 2*np.pi, n_features, endpoint=False).tolist()
        angles += angles[:1]  # 闭合图形
        
        for i, means in enumerate(cluster_means_original):
            # 归一化到0-1范围用于雷达图显示
            normalized_means = (means - means.min()) / (means.max() - means.min() + 1e-8)
            values = normalized_means.tolist()
            values += values[:1]
            
            ax.plot(angles, values, 'o-', linewidth=2, label=f'聚类{i}', markersize=8)
            ax.fill(angles, values, alpha=0.1)
        
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(selected_features)
        ax.set_title('聚类特征雷达图（原始值）', fontsize=14, fontweight='bold', pad=20)
        ax.legend(bbox_to_anchor=(1.1, 1.05))
        ax.grid(True)
        
        plt.tight_layout()
        plt.savefig('../outputs/figures/cluster_radar_chart.png', dpi=300, bbox_inches='tight')
        plt.show()

# 9. 聚类演化分析（跨年度）
print("\n=== 聚类演化分析 ===")

# 分析各年度聚类变化
years = sorted(panel_data['年份'].unique())
yearly_clusters = {}

for year in years:
    year_data = panel_data[panel_data['年份'] == year]
    
    if len(year_data) < 5:  # 数据太少跳过
        continue
    
    # 提取数据
    X_year = year_data[existing_vars].values
    city_labels_year = year_data['城市'].values
    
    # 预处理
    X_year_clean = preprocessor.handle_missing_values(
        pd.DataFrame(X_year, columns=existing_vars)
    )
    X_year_scaled = preprocessor.scale_data(X_year_clean)
    
    # 聚类
    labels_year = cluster_analyzer.fit_clustering(
        X_year_scaled.values,
        method=best_method,
        n_clusters=optimal_n_clusters
    )
    
    yearly_clusters[year] = dict(zip(city_labels_year, labels_year))

print(f"分析了 {len(yearly_clusters)} 个年度的聚类")

# 10. 保存结果
print("\n=== 保存结果 ===")

# 保存聚类结果
cluster_results_df.to_csv('../outputs/tables/clustering_results.csv', index=False)
print("聚类结果已保存至: ../outputs/tables/clustering_results.csv")

# 保存聚类统计
cluster_stats.to_csv('../outputs/tables/cluster_statistics.csv', index=False)
print("聚类统计已保存至: ../outputs/tables/cluster_statistics.csv")

# 保存年度聚类结果
yearly_clusters_df = pd.DataFrame(yearly_clusters)
yearly_clusters_df.to_csv('../outputs/tables/yearly_clusters.csv')
print("年度聚类结果已保存至: ../outputs/tables/yearly_clusters.csv")

# 保存评估分数
kmeans_scores_df = pd.DataFrame(kmeans_scores)
kmeans_scores_df.to_csv('../outputs/tables/cluster_evaluation_scores.csv', index=False)
print("聚类评估分数已保存至: ../outputs/tables/cluster_evaluation_scores.csv")

print("\n=== 聚类分析完成 ===")
print(f"生成图表数量: 4张")
print(f"生成表格数量: 4个")
print(f"最优聚类方法: {best_method}")
print(f"最优聚类数量: {optimal_n_clusters}")
print(f"最佳轮廓系数: {best_score:.3f}")
print(f"所有输出已保存至 ../outputs/ 目录")